In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoNHITS
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import RNN

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelNHITSProcessor:
    def __init__(self, dates=[]):
        self.dates          = dates
        self.forecasts      = []
        self.eval_pairs     = []
        self.input_size     = None
        self.nf             = None
        self.config         = None
        self.testset        = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.reference_dates = {}
        self.metrics_df     = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.metrics_display_df = pd.DataFrame()
        self.display_df     = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df         = pd.DataFrame()
        self.model          = None
        self.selected_input_size = None
        self.unique_ids     = []

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, test_ids, h, freq, level=[]):
        input_len      = df_train.groupby("unique_id").size().min()
        max_input_size = max(8, input_len - h - 1)
        self.testset   = df_test

        def config(trial):
            return {
                "input_size"   : trial.suggest_int("input_size", 8, max_input_size),
                "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"   : trial.suggest_categorical("batch_size", [16, 32, 64]),
                "random_seed"  : trial.suggest_int("random_seed", 1, 99999),
                "accelerator"  : "gpu",
                "devices"      : 2,
                "strategy"     : "auto"
            }

        self.config = config
        if level:
            nf = NeuralForecast(models=[AutoNHITS(h=h, backend="optuna",
                                                  loss=MQLoss(level=level),
                                                  config=self.config)], freq=freq)
        else:
            nf = NeuralForecast(models=[AutoNHITS(h=h, backend="optuna",
                                                  config=self.config)], freq=freq)

        nf.fit(df=df_train)
        self.nf   = nf
        self.model = nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4) -> pd.DataFrame:
        uid_dates = {uid: g["ds"].sort_values().to_numpy()
                     for uid, g in self.testset.groupby("unique_id")}
        T  = min(len(d) for d in uid_dates.values())
        s  = self.selected_input_size
        results = []

        for t in tqdm(range(s, T - h + 1), desc="Batched by index t"):
            batch_records, uid_refs = [], []

            for uid, dates in uid_dates.items():
                ref_date  = dates[t-1]
                win_dates = dates[t-s:t]

                df_series = self.testset[self.testset["unique_id"] == uid]
                df_input  = df_series[df_series["ds"].isin(win_dates)].copy()

                if len(df_input) == s:
                    df_input["unique_id"] = uid
                    batch_records.append(df_input)
                    uid_refs.append((uid, ref_date))

            if not batch_records:
                continue

            batch_df   = pd.concat(batch_records)
            fcst_batch = self.nf.predict(df=batch_df).set_index("ds")

            for uid, ref_date in uid_refs:
                fcast = fcst_batch[fcst_batch["unique_id"] == uid].reset_index()
                gt    = (self.testset[self.testset["unique_id"] == uid]
                         .set_index("ds").loc[fcast["ds"]].reset_index())

                merged = pd.merge(fcast, gt, on=["unique_id", "ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref_date).strftime("%Y-%m-%d")
                results.append(merged)

        return pd.concat(results, ignore_index=True)


    def calculate_metrics(self):
        for fcast, truth in self.eval_pairs:
            y_true = truth.iloc[:,1]
            y_pred = fcast.iloc[:,0]
            self.maes.append(mean_absolute_error(y_true, y_pred))
            self.mses.append(mean_squared_error(y_true, y_pred))
            self.mapes.append(mean_absolute_percentage_error(y_true, y_pred))
            self.nmses.append(self.mses[-1] / np.var(y_true))

    def create_metrics_df(self):
        for i in range(len(self.dates)):
            self.metrics_df.loc[len(self.metrics_df)] = [
                self.unique_ids[i], self.dates[i],
                self.maes[i], self.mses[i], self.mapes[i], self.nmses[i]
            ]

    def create_display_df(self):
        records = []
        idxed = self.testset.set_index(["unique_id","ds"])
        for i, fcast in enumerate(tqdm(self.forecasts, desc="Building display_df")):
            uid = fcast["unique_id"].iloc[0]
            ref = self.dates[i]
            try:   gt_series = idxed.loc[uid]["y"]
            except KeyError: gt_series = pd.Series()

            for col in fcast.columns:
                if col == "unique_id": continue
                if "lo" in col:
                    num = int(col.split("-")[-1]); alpha = 1 - (num/100); q = alpha/2
                elif "hi" in col:
                    num = int(col.split("-")[-1]); alpha = 1 - (num/100); q = 1-alpha/2
                elif col in ["AutoNHITS","AutoNHITS-median"]:
                    q = 0.5
                else: continue

                tmp = fcast[[col]].copy().rename(columns={col:"Prediction"})
                tmp["Unique_id"] = uid
                tmp["Reference Date"] = ref
                tmp["Target End Date"] = tmp.index
                tmp["GT"] = tmp.index.map(gt_series.get)
                tmp["Quantile"] = q
                records.append(tmp)

        self.display_df = (pd.concat(records)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    def calculate_pointwise_metrics(self):
        recs = []
        for i in tqdm(range(len(self.eval_pairs)), desc="Pointwise errors"):
            fcast, truth = self.forecasts[i], self.eval_pairs[i][1]
            ref_date = self.dates[i]; uid = self.unique_ids[i]
            pred_series = fcast["AutoNHITS-median"]; true_series = truth.iloc[:,1]
            for tgt in pred_series.index:
                y_pred = pred_series.loc[tgt]; y_true = true_series.loc[tgt]
                mae = abs(y_pred - y_true)
                mse = (y_pred - y_true)**2
                mape = abs((y_pred - y_true)/y_true) if y_true else np.nan
                nmse = mse / np.var(true_series) if np.var(true_series)!=0 else np.nan
                recs.append({"Unique_id":uid,"Reference Date":ref_date,
                             "Target End Date":tgt,"GT":y_true,
                             "MAE":mae,"MSE":mse,"MAPE":mape,"NMSE":nmse})
        self.metrics_display_df = pd.DataFrame(recs)

    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df() before WIS.")
        df = self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        recs = []
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]; preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            ae = abs(preds[0.5]-gt)
            qs  = sorted(q for q in preds.index if q!=0.5); n=len(qs)//2
            ints = [(preds[qs[i]], preds[qs[-i-1]], qs[-i-1]-qs[i]) for i in range(n)]
            int_score = sum((hi-lo)+(2/a)*max(lo-gt,0)+(2/a)*max(gt-hi,0)
                            for lo,hi,a in ints)
            wis = (ae + int_score) / (1+n)
            recs.append({"Unique_id":uid,"Reference Date":ref,"Target End Date":tgt,
                         "GT":gt,"WIS":wis})
        self.wis_df = pd.DataFrame(recs)

In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelNHITSProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, test_ids, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 03:20:48,671] A new study created in memory with name: no-name-62faa3b1-f51e-43b9-8302-0262231b1985
[rank: 0] Seed set to 89818
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 23     | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [10]:
results = []

In [11]:
processor.selected_input_size

38

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

Batched by index t:   0%|          | 0/19 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:   5%|▌         | 1/19 [00:34<10:24, 34.71s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:  11%|█         | 2/19 [01:07<09:34, 33.80s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

Batched by index t:  89%|████████▉ | 17/19 [09:36<01:07, 33.76s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:  95%|█████████▍| 18/19 [10:08<00:33, 33.33s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t: 100%|██████████| 19/19 [10:41<00:00, 33.78s/it]


In [13]:
all_results

,ds,unique_id,AutoNHITS-median,AutoNHITS-lo-95,AutoNHITS-lo-90,AutoNHITS-lo-85,AutoNHITS-lo-80,AutoNHITS-lo-70,AutoNHITS-lo-60,AutoNHITS-lo-50,...,AutoNHITS-hi-40,AutoNHITS-hi-50,AutoNHITS-hi-60,AutoNHITS-hi-70,AutoNHITS-hi-80,AutoNHITS-hi-85,AutoNHITS-hi-90,AutoNHITS-hi-95,y,Reference Date
0,2011-06-11,Y_1,0.060244,-0.423503,-0.089731,-0.058048,-0.052256,-0.008109,0.000442,0.026734,...,0.062456,0.064240,0.080630,0.084511,0.100285,0.110070,0.130676,0.350635,0.123609,2011-06-04
1,2011-06-18,Y_1,0.060190,-0.476306,-0.097470,-0.057714,-0.052242,-0.008292,0.000463,0.026804,...,0.062995,0.064019,0.080778,0.084230,0.100040,0.109895,0.130232,0.298311,0.051706,2011-06-04
2,2011-06-25,Y_1,0.060176,-0.623017,-0.134241,-0.058244,-0.051764,-0.008248,0.000340,0.026924,...,0.062641,0.064101,0.080706,0.084188,0.100040,0.109929,0.129306,0.229422,0.148478,2011-06-04
3,2011-07-02,Y_1,0.060313,-0.619835,-0.214201,-0.058647,-0.051930,-0.008156,0.000315,0.027080,...,0.062518,0.064084,0.080820,0.083993,0.100130,0.109986,0.129735,0.205300,0.264375,2011-06-04
4,2022-02-05,Y_1000,-50.548771,-1699.274902,-1453.483643,-625.969238,-503.516388,-130.291245,-91.499207,-101.369316,...,-24.075716,17.777702,-29.697750,427.684357,422.721069,756.236450,925.714600,1593.917114,0.000000,2022-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164155,1955-04-16,Y_9998,-0.000004,-0.498401,-0.127692,-0.001599,-0.000514,0.000027,-0.000186,-0.000080,...,-0.000002,0.000166,0.000448,0.000265,0.000133,0.000476,0.000870,0.019775,0.000000,1955-03-19
164156,1957-03-02,Y_9999,1.999928,1.697931,1.996778,1.999000,1.999160,2.000074,1.999942,1.999575,...,1.999935,2.000322,2.000258,2.000784,2.000288,2.000560,2.001811,2.165109,0.000000,1957-02-23
164157,1957-03-09,Y_9999,1.999874,1.645128,1.989039,1.999334,1.999174,1.999891,1.999963,1.999645,...,2.000475,2.000101,2.000406,2.000503,2.000043,2.000384,2.001367,2.112786,0.000000,1957-02-23
164158,1957-03-16,Y_9999,1.999860,1.498416,1.952268,1.998805,1.999652,1.999934,1.999840,1.999765,...,2.000121,2.000183,2.000334,2.000461,2.000043,2.000419,2.000441,2.043897,0.000000,1957-02-23


In [14]:
all_results.to_csv('NHITS_concatenated.csv')

In [3]:
sample = pd.read_csv('NHITS_concatenated.csv')

In [4]:
sample

,Unnamed: 0,ds,unique_id,AutoNHITS-median,AutoNHITS-lo-95,AutoNHITS-lo-90,AutoNHITS-lo-85,AutoNHITS-lo-80,AutoNHITS-lo-70,AutoNHITS-lo-60,...,AutoNHITS-hi-40,AutoNHITS-hi-50,AutoNHITS-hi-60,AutoNHITS-hi-70,AutoNHITS-hi-80,AutoNHITS-hi-85,AutoNHITS-hi-90,AutoNHITS-hi-95,y,Reference Date
0,0,2011-06-11,Y_1,0.060244,-0.423503,-0.089731,-0.058048,-0.052256,-0.008109,0.000442,...,0.062456,0.064240,0.080630,0.084511,0.100285,0.110070,0.130676,0.350635,0.123609,2011-06-04
1,1,2011-06-18,Y_1,0.060190,-0.476305,-0.097470,-0.057714,-0.052242,-0.008292,0.000463,...,0.062995,0.064019,0.080778,0.084230,0.100040,0.109895,0.130232,0.298311,0.051706,2011-06-04
2,2,2011-06-25,Y_1,0.060176,-0.623018,-0.134241,-0.058244,-0.051764,-0.008248,0.000340,...,0.062641,0.064101,0.080706,0.084188,0.100040,0.109929,0.129306,0.229422,0.148478,2011-06-04
3,3,2011-07-02,Y_1,0.060313,-0.619835,-0.214201,-0.058647,-0.051930,-0.008156,0.000315,...,0.062518,0.064084,0.080820,0.083993,0.100130,0.109986,0.129735,0.205300,0.264375,2011-06-04
4,4,2022-02-05,Y_1000,-50.548770,-1699.274900,-1453.483600,-625.969240,-503.516400,-130.291240,-91.499210,...,-24.075716,17.777702,-29.697750,427.684360,422.721070,756.236450,925.714600,1593.917100,0.000000,2022-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164155,164155,1955-04-16,Y_9998,-0.000004,-0.498401,-0.127692,-0.001599,-0.000514,0.000027,-0.000186,...,-0.000002,0.000166,0.000448,0.000265,0.000133,0.000476,0.000870,0.019775,0.000000,1955-03-19
164156,164156,1957-03-02,Y_9999,1.999928,1.697931,1.996778,1.999000,1.999160,2.000074,1.999942,...,1.999935,2.000322,2.000258,2.000784,2.000288,2.000560,2.001811,2.165109,0.000000,1957-02-23
164157,164157,1957-03-09,Y_9999,1.999874,1.645128,1.989039,1.999334,1.999175,1.999891,1.999963,...,2.000475,2.000101,2.000406,2.000503,2.000043,2.000384,2.001367,2.112786,0.000000,1957-02-23
164158,164158,1957-03-16,Y_9999,1.999860,1.498416,1.952268,1.998805,1.999652,1.999934,1.999840,...,2.000121,2.000183,2.000334,2.000461,2.000043,2.000419,2.000441,2.043897,0.000000,1957-02-23


In [5]:
df = sample.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoNHITS-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 164160/164160 [01:14<00:00, 2193.87it/s]


In [6]:
pointwise_df.to_csv('NHITS_ALL_STATS.csv')

In [12]:
wis_table = processor.efficient_compute_wis()

Computing WIS: 100%|██████████| 8640/8640 [00:03<00:00, 2764.91it/s]


In [13]:
wis_table

,Unique_id,Reference Date,Target End Date,GT,WIS
0,Y_1,2011-10-15,2011-10-15,0.0,0.006767
1,Y_1,2011-10-15,2011-10-22,0.0,0.007100
2,Y_1,2011-10-15,2011-10-29,0.0,0.008829
3,Y_1,2011-10-15,2011-11-05,0.0,0.007788
4,Y_1000,2022-06-11,2022-06-11,0.0,0.009985
...,...,...,...,...,...
8635,Y_9998,1955-03-26,1955-04-16,0.0,0.003118
8636,Y_9999,1957-03-02,1957-03-02,0.0,1.185467
8637,Y_9999,1957-03-02,1957-03-09,0.0,0.670586
8638,Y_9999,1957-03-02,1957-03-16,0.0,0.332110


In [14]:
wis_table.to_csv('WIS_NHITS.csv')

In [16]:
mean_wis = np.mean(wis_table['WIS'].values)

In [17]:
mean_wis

2.4806335370188757

In [18]:
wis_dfs = [wis_table.iloc[i::4].reset_index(drop=True) for i in range(4)]

In [22]:
np.mean(wis_dfs[3]['WIS'].values)

2.719905654333445